<a href="https://colab.research.google.com/github/josvaldes/trabajoGradoMCD/blob/develop/scrapingColombiaTicFinal3_ipynb_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#  Notebook 1 - ColombiaTIC: Scraping + Carga Incremental a DuckDB
#  Versión optimizada: conexión segura, control_cargue, y logs persistentes
# ============================================================

# ========== 0. Preparación e instalaciones (auto) ==========
import sys, subprocess, importlib

def pip_install(pkgs):
    for p in pkgs:
        try:
            importlib.import_module(p if p != 'requests_html' else 'requests_html')
        except ImportError:
            print(f"📦 Instalando {p}...")
            subprocess.run([sys.executable, "-m", "pip", "install", p, "-q"], check=True)

# Paquetes requeridos
pip_install(["duckdb", "pandas", "openpyxl", "tqdm", "requests", "requests_html", "lxml_html_clean", "nest_asyncio"])

# ========== 1. Imports y paths ==========
import os, re, shutil, time, logging
from datetime import datetime
import duckdb
import pandas as pd
import requests
from tqdm import tqdm
import nest_asyncio
nest_asyncio.apply()

from requests_html import AsyncHTMLSession
from google.colab import drive

# Rutas base (ajusta si lo necesitas)
RUTA_EXCEL_DEST   = "/content/gdrive/MyDrive/trabajoGrado/reporte_colombiatic"
RUTA_EXCEL_TEMP   = "/content/gdrive/MyDrive/trabajoGrado/temp_colombiatic"
RUTA_DB_DRIVE_DIR = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos"
RUTA_DB_DRIVE     = os.path.join(RUTA_DB_DRIVE_DIR, "colombiatic.duckdb")
RUTA_DB_LOCAL     = "/content/colombiatic_temp.duckdb"     # base temporal local para evitar IO de Drive
RUTA_LOG          = os.path.join(RUTA_DB_DRIVE_DIR, "colombiatic_etl.log")  # LOG persistente junto a la base

os.makedirs(RUTA_EXCEL_DEST, exist_ok=True)
os.makedirs(RUTA_EXCEL_TEMP, exist_ok=True)
os.makedirs(RUTA_DB_DRIVE_DIR, exist_ok=True)

# ============================================================
# 🔗 Montaje robusto de Google Drive
# ============================================================
from google.colab import drive
import os
import shutil

mount_path = "/content/gdrive"

try:
    # Si ya existe y contiene archivos, lo desmontamos primero
    if os.path.exists(mount_path) and os.listdir(mount_path):
        print("⚙️  Desmontando Drive previo...")
        drive.flush_and_unmount()
        shutil.rmtree(mount_path, ignore_errors=True)

    # Montar Drive limpio
    drive.mount(mount_path, force_remount=True)
    print("✅ Google Drive montado correctamente en:", mount_path)
except Exception as e:
    print(f"⚠️ Error al montar Google Drive: {e}")



# ========== 2. Logger persistente ==========
logger = logging.getLogger("colombiatic_etl")
logger.setLevel(logging.INFO)
# Evitar handlers duplicados en re-ejecuciones
if not logger.handlers:
    fh = logging.FileHandler(RUTA_LOG, encoding="utf-8")
    fh.setLevel(logging.INFO)
    fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")
    fh.setFormatter(fmt)
    logger.addHandler(fh)

def log_info(msg):
    print(msg)
    logger.info(msg)

def log_warn(msg):
    print(msg)
    logger.warning(msg)

def log_error(msg):
    print(msg)
    logger.error(msg)

log_info("===== INICIO EJECUCIÓN NOTEBOOK 1 (Scraping + ETL) =====")
log_info(f"Carpeta destino Excel: {RUTA_EXCEL_DEST}")
log_info(f"Carpeta temporal Excel: {RUTA_EXCEL_TEMP}")
log_info(f"Base en Drive: {RUTA_DB_DRIVE}")
log_info(f"Log persistente: {RUTA_LOG}")

# ========== 3. Utilidades (nombres y conexión) ==========
def normalizar_nombre(s: str, max_len: int = 60) -> str:
    s = s.lower()
    s = re.sub(r"[^a-z0-9_]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s[:max_len] if max_len else s

def conectar_duckdb_seguro(path_db: str):
    """Conecta o crea base. Si existe en Drive, copia a local y conecta local."""
    # Si existe base en Drive, hacer copia local para trabajar rápido/sin locks
    if os.path.exists(RUTA_DB_DRIVE):
        try:
            if os.path.exists(RUTA_DB_LOCAL):
                os.remove(RUTA_DB_LOCAL)
            shutil.copy2(RUTA_DB_DRIVE, RUTA_DB_LOCAL)
            log_info(f"✅ Copia local creada desde Drive: {RUTA_DB_DRIVE} → {RUTA_DB_LOCAL}")
        except Exception as e:
            log_warn(f"⚠️ No se pudo copiar base desde Drive. Se creará una nueva local. Detalle: {e}")
    # Conectar a la copia local (o nueva si no existía)
    con = duckdb.connect(RUTA_DB_LOCAL)
    log_info(f"💾 Conexión establecida con base local: {RUTA_DB_LOCAL}")
    # Tabla de control (si no existe)
    con.execute("""
    CREATE TABLE IF NOT EXISTS control_cargue (
        archivo TEXT,
        hojas_cargadas INTEGER,
        filas_totales INTEGER,
        fecha_cargue TIMESTAMP,
        estado TEXT
    )
    """)
    log_info("📊 Tabla 'control_cargue' lista.")
    return con

def sincronizar_local_a_drive():
    """Cierra conexión si está abierta y copia local → Drive."""
    try:
        con.close()
    except:
        pass
    try:
        if os.path.exists(RUTA_DB_LOCAL):
            shutil.copy2(RUTA_DB_LOCAL, RUTA_DB_DRIVE)
            log_info(f"📦 Base actualizada copiada de nuevo al Drive: {RUTA_DB_LOCAL} → {RUTA_DB_DRIVE}")
    except Exception as e:
        log_error(f"❌ Error copiando base a Drive: {e}")

# ========== 4. Scraping (encuentra y descarga solo Excel; filtra TIC) ==========
URL_PORTAL = "https://colombiatic.mintic.gov.co/679/w3-channel.html"

async def obtener_urls_excel():
    """Rastrea artículos del canal y devuelve URLs .xlsx únicas."""
    session = AsyncHTMLSession()
    try:
        r = await session.get(URL_PORTAL)
        await r.html.arender(timeout=60, sleep=2)
        # enlances a artículos del canal
        links = r.html.absolute_links
        urls_articulos = [u for u in links if "/w3-article-" in u or "/articles-" in u]
        urls_articulos = list(dict.fromkeys(urls_articulos))  # únicos
        log_info(f"📰 Artículos detectados: {len(urls_articulos)}")

        urls_xlsx = set()
        for u in tqdm(urls_articulos, desc="🔎 Explorando artículos"):
            try:
                a = await session.get(u)
                await a.html.arender(timeout=60, sleep=1)
                for l in a.html.absolute_links:
                    if l.lower().endswith(".xlsx") or "archivo_xls.xlsx" in l.lower():
                        urls_xlsx.add(l)
            except Exception as e:
                log_warn(f"⚠️ Error explorando {u}: {e}")

        return list(urls_xlsx)
    finally:
        await session.close()

def descargar_excel(url: str, destino_dir: str) -> str | None:
    """Descarga un Excel y lo nombra a partir del slug del artículo o filename; retorna ruta destino."""
    try:
        resp = requests.get(url, timeout=60)
        if resp.status_code != 200:
            log_warn(f"⚠️ No se pudo descargar {url} (status {resp.status_code})")
            return None
        # nombre base por defecto
        base = url.split("/")[-1].replace(".xlsx", "")
        base = normalizar_nombre(base, 80)

        # guardar en temp
        nombre = f"{base}.xlsx"
        ruta = os.path.join(destino_dir, nombre)
        with open(ruta, "wb") as f:
            f.write(resp.content)
        return ruta
    except Exception as e:
        log_warn(f"⚠️ Error descargando {url}: {e}")
        return None

# Lanzar scraping
log_info("🌐 Iniciando scraping de boletines...")
try:
    # Ejecutar asincrónicamente en Colab
    import asyncio
    urls_xlsx = asyncio.get_event_loop().run_until_complete(obtener_urls_excel())
except RuntimeError:
    # Loop ya corriendo: usar nest_asyncio y crear uno nuevo
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    urls_xlsx = loop.run_until_complete(obtener_urls_excel())

log_info(f"📊 Archivos Excel detectados (únicos): {len(urls_xlsx)}")

# Descarga a carpeta temporal
descargados = []
for u in tqdm(urls_xlsx, desc="⬇️ Descargando archivos Excel"):
    ruta_tmp = descargar_excel(u, RUTA_EXCEL_TEMP)
    if ruta_tmp:
        descargados.append(ruta_tmp)

log_info(f"📦 Archivos descargados en temp: {len(descargados)}")

# Detectar TIC y mover a carpeta final. Borrar los no TIC
movidos = []
eliminados = []
for ruta in descargados:
    nombre = os.path.basename(ruta)
    # Heurística: debe contener "tic" en el nombre (en muchos casos funciona),
    # y/o es el archivo principal que el portal publica (ya trae "sector_tic" o similar).
    if "tic" in nombre.lower():
        destino = os.path.join(RUTA_EXCEL_DEST, nombre)
        if os.path.exists(destino):
            os.remove(destino)
        shutil.move(ruta, destino)
        movidos.append(destino)
        log_info(f"📦 Movido a carpeta destino: {os.path.basename(destino)}")
    else:
        os.remove(ruta)
        eliminados.append(nombre)
        log_info(f"🧹 Eliminado archivo no relevante: {nombre}")

# Limpieza final de temp (si quedó algo)
try:
    for f in os.listdir(RUTA_EXCEL_TEMP):
        try:
            os.remove(os.path.join(RUTA_EXCEL_TEMP, f))
        except:
            pass
except:
    pass

log_info("🎯 Scraping finalizado.")

# ========== 5. Conexión a DuckDB (local) ==========
# Importante: conectar después del scraping
con = conectar_duckdb_seguro(RUTA_DB_DRIVE)

# ========== 6. Carga incremental de Excel TIC ==========
def archivos_tic_actuales():
    return [f for f in os.listdir(RUTA_EXCEL_DEST) if f.lower().endswith((".xlsx", ".xls"))]

# Obtén los ya cargados desde control_cargue
try:
    ya_cargados = set(
        con.execute("SELECT DISTINCT archivo FROM control_cargue").fetchdf()["archivo"].tolist()
    )
except Exception:
    ya_cargados = set()

archivos = archivos_tic_actuales()
pendientes = [f for f in archivos if f not in ya_cargados]

print("\n📂 Archivos disponibles actualmente en carpeta destino:")
for f in archivos:
    print(f"   • {f}")

if not pendientes:
    log_info("⏭️ No hay archivos nuevos para cargar. La base está actualizada.")
else:
    log_info(f"🆕 Archivos TIC nuevos detectados: {len(pendientes)}")

for nombre_archivo in pendientes:
    ruta_archivo = os.path.join(RUTA_EXCEL_DEST, nombre_archivo)
    estado = "OK"
    total_filas = 0
    hojas_cargadas = 0
    log_info(f"\n📘 Procesando: {nombre_archivo}")

    try:
        xls = pd.ExcelFile(ruta_archivo)
        hojas = xls.sheet_names
        # proceso hoja a hoja
        for hoja in hojas:
            try:
                # Detectar encabezado: primeras 10 filas probando
                df_valid = None
                for fila_enc in range(0, 10):
                    df_try = pd.read_excel(ruta_archivo, sheet_name=hoja, header=fila_enc, engine="openpyxl")
                    if df_try.columns.notna().sum() > 2:
                        df_valid = df_try
                        break
                if df_valid is None or df_valid.empty:
                    continue

                df_valid = df_valid.astype(str)

                # nombre de tabla
                base = normalizar_nombre(os.path.splitext(nombre_archivo)[0], 48)
                hoja_n = normalizar_nombre(hoja, 10)  # corto para no pasarse de 63
                nombre_tabla = f"{base}_{hoja_n}"

                # Evitar duplicados de tabla
                tablas_existentes = con.execute("SHOW TABLES").fetchdf()["name"].tolist()
                if nombre_tabla in tablas_existentes:
                    log_info(f"   ⏭️ La tabla '{nombre_tabla}' ya existe, se omite.")
                else:
                    con.register("tmp_df", df_valid)
                    con.execute(f"CREATE TABLE '{nombre_tabla}' AS SELECT * FROM tmp_df")
                    con.unregister("tmp_df")
                    hojas_cargadas += 1
                    total_filas += len(df_valid)
                    log_info(f"   ✅ Hoja '{hoja}' cargada como '{nombre_tabla}' ({len(df_valid)} filas)")
            except Exception as e_hoja:
                estado = f"ERROR_HOJA:{hoja}:{str(e_hoja)[:150]}"
                log_warn(f"⚠️ {estado}")

    except Exception as e_file:
        estado = f"ERROR_ARCHIVO:{str(e_file)[:150]}"
        log_warn(f"⚠️ {estado}")

    # Registrar cargue a nivel archivo (incremental)
    try:
        con.execute("""
            INSERT INTO control_cargue (archivo, hojas_cargadas, filas_totales, fecha_cargue, estado)
            VALUES (?, ?, ?, ?, ?)
        """, [nombre_archivo, hojas_cargadas, total_filas, datetime.now(), estado])
        log_info(f"📝 Registrado en control_cargue: {nombre_archivo} | hojas={hojas_cargadas} | filas={total_filas} | estado={estado}")
    except Exception as e_ins:
        log_warn(f"⚠️ Error insertando en control_cargue: {e_ins}")

# Sincronizar base local a Drive y cerrar
sincronizar_local_a_drive()

# ========== 7. Revisar la base (resumen final) ==========
# Para evitar el error de conexión cerrada, abrimos una conexión **ligera** a la base en Drive solo para consulta
try:
    con_check = duckdb.connect(RUTA_DB_DRIVE, read_only=True)
    tablas = con_check.execute("SHOW TABLES").fetchdf()
    print("\n📂 Tablas en la base:")
    display(tablas)

    control = con_check.execute("""
        SELECT archivo, hojas_cargadas, filas_totales, fecha_cargue, estado
        FROM control_cargue
        ORDER BY fecha_cargue DESC
        LIMIT 20
    """).fetchdf()
    print("\n📋 Últimos registros en control_cargue:")
    display(control)

    total_control = con_check.execute("SELECT COUNT(*) AS total FROM control_cargue").fetchdf()
    print("\n🧮 Total de registros en control_cargue:")
    display(total_control)

    # tamaño del archivo de base
    tam_mb = os.path.getsize(RUTA_DB_DRIVE) / (1024*1024)
    print(f"\n💾 Tamaño actual de la base: {tam_mb:.2f} MB")
    con_check.close()
except Exception as e:
    log_warn(f"⚠️ No se pudo consultar la base final: {e}")

log_info("===== FIN EJECUCIÓN NOTEBOOK 1 =====")
print("\n✅ Proceso completado.")
print(f"🗂 Log del ETL: {RUTA_LOG}")
print(f"🗄️ Base DuckDB: {RUTA_DB_DRIVE}")
print(f"📁 Carpeta Excel (solo TIC): {RUTA_EXCEL_DEST}")


In [ ]:
import duckdb, pandas as pd

# Conexión (solo lectura)
db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path, read_only=True)

# Listar todas las tablas que contengan "_4_1"
tablas_41 = con.execute("""
    SELECT table_name AS name
    FROM duckdb_tables()
    WHERE table_name LIKE '%4_1%'
    ORDER BY table_name
""").fetchdf()

print("📋 Tablas 4_1 detectadas:")
display(tablas_41)



Bloque de código final para consolidar automáticamente

In [ ]:
import duckdb, pandas as pd

# --- Asegurar conexión limpia ---
try:
    con.close()
except:
    pass

db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)
print("✅ Conectado a la base correctamente.\n")

# --- Buscar todas las tablas que terminen en _4_1 ---
tablas_41 = con.execute("""
    SELECT table_name AS name
    FROM duckdb_tables()
    WHERE table_name LIKE '%4_1'
    ORDER BY table_name
""").fetchdf()

if tablas_41.empty:
    print("⚠️ No hay tablas 4.1 disponibles.")
else:
    print(f"📋 Tablas detectadas para consolidar ({len(tablas_41)}):")
    display(tablas_41)

    # --- Obtener todas las columnas únicas entre las tablas 4.1 ---
    columnas_union = set()
    columnas_por_tabla = {}

    for t in tablas_41["name"]:
        cols = con.execute(f"PRAGMA table_info('{t}')").fetchdf()["name"].tolist()
        columnas_union.update(cols)
        columnas_por_tabla[t] = cols

    columnas_union = sorted(list(columnas_union))
    print(f"🧱 Total columnas únicas detectadas: {len(columnas_union)}\n")

    # --- Crear sentencias SELECT alineadas ---
    selects = []
    for t, cols in columnas_por_tabla.items():
        # agregar NULL para columnas faltantes
        select_cols = [f"'{t}' AS origen"]
        for col in columnas_union:
            if col in cols:
                select_cols.append(f'"{col}"')
            else:
                select_cols.append(f"NULL AS \"{col}\"")
        selects.append(f"SELECT {', '.join(select_cols)} FROM '{t}'")

    union_query = " UNION ALL ".join(selects)

    # --- Crear tabla consolidada ---
    con.execute(f"""
        CREATE OR REPLACE TABLE consolidado_tic_4_1 AS {union_query}
    """)
    print("✅ Tabla consolidada creada: consolidado_tic_4_1")

    # --- Mostrar resumen de consolidación ---
    resumen = con.execute("""
        SELECT origen, COUNT(*) AS registros
        FROM consolidado_tic_4_1
        GROUP BY origen
        ORDER BY origen
    """).fetchdf()

    print("\n📊 Registros por tabla origen:")
    display(resumen)

    total = con.execute("SELECT COUNT(*) FROM consolidado_tic_4_1").fetchone()[0]
    print(f"🧮 Total de registros consolidados: {total:,}")

con.close()
print("\n🔚 Conexión cerrada correctamente.")


Bloque para explorar y visualizar la tabla consolidada

Código optimizado para limpiar y renombrar consolidado_tic_4_1

In [ ]:
import duckdb, pandas as pd

# --- Cerrar conexión previa si está abierta ---
try:
    con.close()
    print("🔒 Conexión anterior cerrada correctamente.")
except:
    print("ℹ️ No había conexión previa activa.")

# --- Reconexión a la base ---
db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)
print(f"✅ Conectado a la base: {db_path}")

# --- Leer muestra para detectar encabezado correcto ---
df_preview = con.execute("""
    SELECT * FROM consolidado_tic_4_1
    LIMIT 10
""").fetchdf()

print("\n👀 Vista previa actual:")
display(df_preview)

# --- Extraer nombres reales desde la fila 5 (índice 4) ---
header_row = df_preview.iloc[4, 1:].tolist()
header_row = [h.strip().upper() if isinstance(h, str) else f"COLUMN_{i}" for i, h in enumerate(header_row)]

print("\n🧭 Encabezados detectados:")
print(header_row)

# --- Cargar todo el dataset saltando las filas de metadatos (primeras 5 filas) ---
df = con.execute("""
    SELECT * FROM consolidado_tic_4_1
""").fetchdf().iloc[5:].copy()

# --- Asignar nombres de columnas ---
df.columns = ["origen"] + header_row

# --- Limpiar filas vacías o irrelevantes ---
df = df.dropna(subset=["AÑO", "DEPARTAMENTO", "MUNICIPIO"], how="any")

# --- Convertir tipos numéricos ---
cols_numericas = ["AÑO", "TRIMESTRE", "VELOCIDAD SUBIDA", "VELOCIDAD BAJADA", "No. ACCESOS FIJOS A INTERNET"]
for col in cols_numericas:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# --- Guardar tabla limpia ---
con.register("df_clean", df)
con.execute("CREATE OR REPLACE TABLE consolidado_tic_4_1_limpia AS SELECT * FROM df_clean")
con.unregister("df_clean")

print("\n✅ Tabla 'consolidado_tic_4_1_limpia' creada correctamente.")

# --- Ver estructura y muestra ---
estructura = con.execute("PRAGMA table_info('consolidado_tic_4_1_limpia')").fetchdf()
print("\n📋 Nueva estructura:")
display(estructura)

muestra = con.execute("SELECT * FROM consolidado_tic_4_1_limpia LIMIT 10").fetchdf()
print("\n👀 Vista previa de datos limpios:")
display(muestra)

# --- Cerrar conexión ---
con.close()
print("\n🔚 Conexión cerrada correctamente.")


Validación de cobertura y consistencia

In [ ]:
import duckdb, pandas as pd

db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path, read_only=True)

print("✅ Conectado a la base para análisis exploratorio.\n")

# --- 1️⃣ Departamentos únicos y conteo ---
print("📍 Departamentos y número de municipios reportados:")
deptos = con.execute("""
    SELECT DEPARTAMENTO, COUNT(DISTINCT MUNICIPIO) AS municipios_unicos, COUNT(*) AS total_registros
    FROM consolidado_tic_4_1_limpia
    GROUP BY DEPARTAMENTO
    ORDER BY DEPARTAMENTO
""").fetchdf()
display(deptos)

# --- 2️⃣ Cobertura temporal ---
print("\n🗓️ Años y trimestres disponibles:")
periodos = con.execute("""
    SELECT AÑO, TRIMESTRE, COUNT(*) AS registros
    FROM consolidado_tic_4_1_limpia
    GROUP BY AÑO, TRIMESTRE
    ORDER BY AÑO, TRIMESTRE
""").fetchdf()
display(periodos)

# --- 3️⃣ Validación de velocidades ---
print("\n📈 Rango de velocidades detectadas:")
velocidades = con.execute("""
    SELECT
        MIN("VELOCIDAD SUBIDA") AS min_subida,
        MAX("VELOCIDAD SUBIDA") AS max_subida,
        MIN("VELOCIDAD BAJADA") AS min_bajada,
        MAX("VELOCIDAD BAJADA") AS max_bajada,
        ROUND(AVG("VELOCIDAD SUBIDA"),2) AS promedio_subida,
        ROUND(AVG("VELOCIDAD BAJADA"),2) AS promedio_bajada
    FROM consolidado_tic_4_1_limpia
""").fetchdf()
display(velocidades)

con.close()
print("\n🔚 Conexión cerrada correctamente.")


In [ ]:
import duckdb
con = duckdb.connect("/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb")
con.execute("SHOW TABLES").fetchdf()


Verificación de unicidad

In [ ]:
import duckdb
con = duckdb.connect("/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb")

duplicados = con.execute("""
    SELECT COUNT(*) AS total_registros,
           COUNT(DISTINCT (AÑO, TRIMESTRE, DEPARTAMENTO, MUNICIPIO, PROVEEDOR, "TECNOLOGÍA")) AS registros_unicos
    FROM consolidado_tic_4_1_limpia
""").fetchdf()

print(duplicados)
con.close()


Consulta para verificar dimensiones y muestra de datos

In [ ]:
import duckdb
import pandas as pd

# --- Conexión a la base ---
db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)

# --- 1️⃣ Dimensiones de la tabla ---
dim = con.execute("""
    SELECT
        COUNT(*) AS total_registros,
        COUNT(DISTINCT DEPARTAMENTO) AS departamentos,
        COUNT(DISTINCT MUNICIPIO) AS municipios,
        COUNT(DISTINCT AÑO) AS anios,
        COUNT(DISTINCT TRIMESTRE) AS trimestres,
        COUNT(DISTINCT PROVEEDOR) AS proveedores,
        COUNT(DISTINCT "TECNOLOGÍA") AS tecnologias
    FROM consolidado_tic_4_1_limpia
""").fetchdf()

print("📏 Dimensiones de la tabla `consolidado_tic_4_1_limpia`:")
display(dim)

# --- 2️⃣ Vista previa ordenada por año y trimestre ---
muestra = con.execute("""
    SELECT AÑO, TRIMESTRE, DEPARTAMENTO, MUNICIPIO, PROVEEDOR,
           "SEGMENTO", "TECNOLOGÍA", "VELOCIDAD SUBIDA", "VELOCIDAD BAJADA",
           "NO. ACCESOS FIJOS A INTERNET"
    FROM consolidado_tic_4_1_limpia
    WHERE AÑO IS NOT NULL
    ORDER BY AÑO DESC, TRIMESTRE DESC
    LIMIT 20
""").fetchdf()

print("\n👀 Vista previa de registros representativos:")
display(muestra)

con.close()
print("\n🔚 Conexión cerrada correctamente.")


Bloque de detección y limpieza de outliers

In [ ]:
import duckdb
import pandas as pd
import numpy as np

db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)

# --- Cargar los datos a memoria para depuración ---
df = con.execute("SELECT * FROM consolidado_tic_4_1_limpia").fetchdf()

print(f"📊 Registros iniciales: {len(df):,}")

# --- Convertir a numérico por seguridad ---
for col in ["VELOCIDAD SUBIDA", "VELOCIDAD BAJADA"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# --- Calcular IQR para cada variable ---
def limpiar_outliers_iqr(serie):
    q1 = serie.quantile(0.25)
    q3 = serie.quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr
    return serie.clip(lower=limite_inferior, upper=limite_superior)

df["VELOCIDAD SUBIDA_LIMPIA"] = limpiar_outliers_iqr(df["VELOCIDAD SUBIDA"])
df["VELOCIDAD BAJADA_LIMPIA"] = limpiar_outliers_iqr(df["VELOCIDAD BAJADA"])

# --- Conteo de valores corregidos ---
cambios_subida = (df["VELOCIDAD SUBIDA"] != df["VELOCIDAD SUBIDA_LIMPIA"]).sum()
cambios_bajada = (df["VELOCIDAD BAJADA"] != df["VELOCIDAD BAJADA_LIMPIA"]).sum()

print(f"⚙️ Registros ajustados (subida): {cambios_subida:,}")
print(f"⚙️ Registros ajustados (bajada): {cambios_bajada:,}")

# --- Guardar tabla limpia ---
con.execute("DROP TABLE IF EXISTS consolidado_tic_4_1_filtrado")
con.register("df_temp", df)
con.execute("""
    CREATE TABLE consolidado_tic_4_1_filtrado AS
    SELECT
        origen, AÑO, TRIMESTRE, DEPARTAMENTO, MUNICIPIO, PROVEEDOR,
        SEGMENTO, TECNOLOGÍA,
        "NO. ACCESOS FIJOS A INTERNET",
        "CÓDIGO DANE", "CÓDIGO DANE_1",
        "VELOCIDAD SUBIDA_LIMPIA" AS VELOCIDAD_SUBIDA,
        "VELOCIDAD BAJADA_LIMPIA" AS VELOCIDAD_BAJADA
    FROM df_temp
""")
con.unregister("df_temp")

# --- Verificar dimensiones ---
dim = con.execute("SELECT COUNT(*) AS total, COUNT(DISTINCT MUNICIPIO) AS municipios FROM consolidado_tic_4_1_filtrado").fetchdf()
print("\n✅ Tabla `consolidado_tic_4_1_filtrado` creada correctamente:")
display(dim)

con.close()
print("\n🔚 Conexión cerrada correctamente.")
